In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import time

In [ ]:
# job_title = "data analyst"
# location = "New York"

In [ ]:
# list_url= "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%2Banalyst&location=United%2BStates&start=150"
# # list_url= "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=United%2BStates&start=25"
# response= requests.get(list_url)
# requests.get(list_url, headers = {'User-agent': 'Chrome/129.0.6668.101'})
# print(response.status_code)
# list_soup= BeautifulSoup(response.text, "html.parser")
# page_jobs= list_soup.find_all("li")

In [ ]:
# len(page_jobs)

In [ ]:
# page_jobs[0]

In [ ]:
# job_id_list=[]

In [ ]:
# for job in page_jobs:
#     base_card_div= job.find("div", {"class": "base-card"})
#     job_id= base_card_div.get("data-entity-urn").split(":")[3]
#     print(job_id)
#     job_id_list.append(job_id)
    

In [ ]:
# len(job_id_list)

In [ ]:
# job_id_list_dedup = list(set(job_id_list))
# len(job_id_list_dedup)

In [ ]:
# pd.DataFrame(job_id_list_dedup).to_csv("job_id_list.csv")

In [2]:
job_id_list = []
total_jobs_to_scrape = 200  
jobs_per_page = 25  
start = 0
# base_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=United%2BStates&start="
# base_url= "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%2Banalyst&location=United%2BStates&start="
base_url= "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=web%2Bdeveloper&location=United%2BStates&start="


In [3]:
while len(job_id_list) < total_jobs_to_scrape:
    list_url = f"{base_url}{start}"
    success = False
    while not success:
        response = requests.get(list_url, headers={'User-agent': 'Chrome/129.0.6668.101'})
        print(f"Scraping jobs starting at {start}, Status Code: {response.status_code}")
        
        if response.status_code == 200:
            success = True
        else:
            print(f"Failed to retrieve data for start={start}, retrying...")
            time.sleep(2)  # Wait before retrying
            
    list_soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all job listings on the current page
    page_jobs = list_soup.find_all("li")
    
    # Loop through each job and extract the job ID
    for job in page_jobs:
        base_card_div = job.find("div", {"class": "base-card"})
        if base_card_div:
            job_id = base_card_div.get("data-entity-urn").split(":")[3]
            job_id_list.append(job_id)
            print(f"Scraped job ID: {job_id}")
            
        # Stop if we've reached the desired count
        if len(job_id_list) >= total_jobs_to_scrape:
            break
    
    # Move to the next page
    start += jobs_per_page

Scraping jobs starting at 0, Status Code: 429
Failed to retrieve data for start=0, retrying...
Scraping jobs starting at 0, Status Code: 200
Scraped job ID: 3940392184
Scraped job ID: 4032889617
Scraped job ID: 4069245024
Scraped job ID: 3625991287
Scraped job ID: 3625989459
Scraped job ID: 4059300826
Scraped job ID: 4037252893
Scraped job ID: 4058442705
Scraped job ID: 4071686693
Scraped job ID: 4007046613
Scraping jobs starting at 25, Status Code: 429
Failed to retrieve data for start=25, retrying...
Scraping jobs starting at 25, Status Code: 200
Scraped job ID: 4068293317
Scraped job ID: 4068294050
Scraped job ID: 3904986456
Scraped job ID: 4004862143
Scraped job ID: 3946663338
Scraped job ID: 4069147539
Scraped job ID: 4041203435
Scraped job ID: 4041206242
Scraped job ID: 4067132721
Scraped job ID: 4026783520
Scraping jobs starting at 50, Status Code: 200
Scraped job ID: 4045410988
Scraped job ID: 4052032064
Scraped job ID: 3972261538
Scraped job ID: 4062204421
Scraped job ID: 4046

In [ ]:
# job_id_list = job_id_list[:total_jobs]

In [4]:
len(job_id_list)

200

In [5]:
jobs_list= []
for job_id in job_id_list:
    job_url= f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    while True:
        job_response= requests.get(job_url, headers = {'User-agent': 'Chrome/129.0.6668.101'})
#         print(job_response.status_code)
        if job_response.status_code == 200:
            print(job_response.status_code)
            break
        elif job_response.status_code == 429:
            time.sleep(int(3))
            continue
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    job_post={}
    
    try:
        job_post["company_name"]= job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post["company_name"]= None
        
    try:
        job_post["job_title"]= job_soup.find("h2", {"class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
    except:
        job_post["job_title"]= None
        
    try:
        job_post["posted_time"]= job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job_post["posted_time"]= None 
        
    try:
        job_post["level"]= job_soup.find("span", {"class": "description__job-criteria-text description__job-criteria-text--criteria"}).text.strip()
    except:
        job_post["level"]= None 
    
    try:
        job_post["type"]= job_soup.find("span", {"class": "description__job-criteria-text description__job-criteria-text--criteria"}).text.strip()
    except:
        job_post["type"]= None 

    try:
        job_post["salary_range"]= job_soup.find("div", {"class": "salary compensation__salary"}).text.strip()
    except:
        job_post["salary_range"]= None
        
    try:
        job_post["requirements"]= job_soup.find("div", {"class": "show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden"}).text.strip()
    except:
        job_post["requirements"]= None
    
    jobs_list.append(job_post)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


ConnectionError: HTTPSConnectionPool(host='www.linkedin.com', port=443): Max retries exceeded with url: /jobs-guest/jobs/api/jobPosting/4035069048 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000027FBC1EABD0>: Failed to resolve 'www.linkedin.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
# jobs_list= []
# for job in jobs:
# #     job_url= f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
# #     job_response= requests.get(job_url)
# #     print(job_response.status_code)
#     job_soup = BeautifulSoup(job_response.text, "html.parser")
#     job_post={}
    
#     try:
#         job_post["company_name"]= job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
#     except:
#         job_post["company_name"]= None
        
#     try:
#         job_post["job_title"]= job_soup.find("h2", {"class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
#     except:
#         job_post["job_title"]= None
        
#     try:
#         job_post["posted_time"]= job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
#     except:
#         job_post["posted_time"]= None 
        
#     try:
#         job_post["level"]= job_soup.find("span", {"class": "description__job-criteria-text description__job-criteria-text--criteria"}).text.strip()
#     except:
#         job_post["level"]= None 
    
#     try:
#         job_post["type"]= job_soup.find("span", {"class": "description__job-criteria-text description__job-criteria-text--criteria"}).text.strip()
#     except:
#         job_post["type"]= None 

#     try:
#         job_post["salary_range"]= job_soup.find("div", {"class": "salary compensation__salary"}).text.strip()
#     except:
#         job_post["salary_range"]= None
        
#     try:
#         job_post["requirements"]= job_soup.find("div", {"class": "show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden"}).text.strip()
#     except:
#         job_post["requirements"]= None     
    
#     jobs_list.append(job_post)       

In [ ]:
jobs_df= pd.DataFrame(jobs_list)

In [ ]:
jobs_df

In [ ]:
jobs_df.to_csv("jobsDA.csv")

In [ ]:
jobs_df.shape